In [2]:
import glob
import pandas as pd

def filter_and_validate_dates(file_pattern, starting_date, ending_date, segment, index):
    # Loading all CSV files matching the pattern
    csv_files = glob.glob(file_pattern)
    dataframes = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df.rename(columns={"Date": "date"}, inplace=True)
            df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")
            dataframes.append(df)
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    # Combining all dataframes into one
    combined_df = pd.concat(dataframes, ignore_index=True)
    combined_df['date'] = pd.to_datetime(combined_df['date'])

    # Generating all dates in the range
    all_dates = pd.date_range(start=starting_date, end=ending_date, freq='D')

    # Filtering data by date range and ticker
    filtered_by_date_range = combined_df[(combined_df['date'] >= pd.to_datetime(starting_date)) & 
                                         (combined_df['date'] <= pd.to_datetime(ending_date))]
    filtered_by_ticker = filtered_by_date_range[
        filtered_by_date_range['Ticker'].str.contains(segment, case=False) & 
        filtered_by_date_range['Ticker'].str.contains(index, case=False)]

    # Validating each date in the range
    date_exists = {date.strftime("%Y-%m-%d"): date in filtered_by_date_range['date'].values for date in all_dates}

    return filtered_by_ticker, date_exists

# Inputs
file_pattern = "*_BACKADJUSTED_*.csv"
starting_date = input("Enter the starting date (YYYY-MM-DD): ")
ending_date = input("Enter the ending date (YYYY-MM-DD): ")
segment = input("Enter the segment (e.g., BFO/NFO/MCX): ")
index = input("Enter the index (e.g., CRUDEOIL, GOLD): ")



Enter the starting date (YYYY-MM-DD): 2023-04-03
Enter the ending date (YYYY-MM-DD): 2023-04-30
Enter the segment (e.g., BFO/NFO/MCX): MCX
Enter the index (e.g., CRUDEOIL, GOLD): CRUDEOIL


Data Filtering

In [3]:
# Function execution to get Filtered Data
data, date_check = filter_and_validate_dates(file_pattern, starting_date, ending_date, segment, index)

# Creating a list of dates where data exists
existing_dates = [date for date, exists in date_check.items() if exists]


In [13]:
# Function execution to get Filtered Data
data, date_check = filter_and_validate_dates(file_pattern, starting_date, ending_date, segment, index)

print(data)
print("Date Existence Check:")
for date, exists in date_check.items():
    print(f"{date}: {'Exists' if exists else 'Does not exist'}")

                         Ticker       date      Time   Close    Spot  Open  \
0       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:02:59     3.0     0.0   NaN   
1       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:22:59     2.9     0.0   NaN   
2       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:26:59     2.7     0.0   NaN   
3       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:46:59     2.8     0.0   NaN   
4       CRUDEOIL23APR4500PE.MCX 2023-04-12  09:53:59     2.8     0.0   NaN   
...                         ...        ...       ...     ...     ...   ...   
538508        CRUDEOILM-III.MCX 2023-04-17  23:06:59  6667.0  6700.0   NaN   
538509        CRUDEOILM-III.MCX 2023-04-17  23:09:59  6663.0  6700.0   NaN   
538510        CRUDEOILM-III.MCX 2023-04-17  23:16:59  6670.0  6700.0   NaN   
538511        CRUDEOILM-III.MCX 2023-04-17  23:17:59  6671.0  6700.0   NaN   
538512        CRUDEOILM-III.MCX 2023-04-17  23:27:59  6663.0  6700.0   NaN   

        High  Low  Volume  Open Interest  Unnamed: 9  Unnamed: 

In [14]:
# Creating a list of dates where data exists
existing_dates = [date for date, exists in date_check.items() if exists]

# Printing the list of existing dates
print("Existing Dates:")
print(existing_dates)


Existing Dates:
['2023-04-03', '2023-04-04', '2023-04-05', '2023-04-06', '2023-04-10', '2023-04-11', '2023-04-12', '2023-04-13', '2023-04-14', '2023-04-17', '2023-04-18', '2023-04-19', '2023-04-20', '2023-04-21', '2023-04-24', '2023-04-25', '2023-04-26', '2023-04-27', '2023-04-28']


In [9]:
from datetime import datetime
import pandas as pd
import re  

months = {
    "JAN": 1, "FEB": 2, "MAR": 3, "APR": 4,
    "MAY": 5, "JUN": 6, "JUL": 7, "AUG": 8,
    "SEP": 9, "OCT": 10, "NOV": 11, "DEC": 12
}


# Assuming this is the function from previous examples
def get_all_expiry_dates(data, input_date, index, segment):
    input_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
    year = input_date_obj.year
    regex_pattern = f"{index}(\d{{2}}).*?\.{segment}"
    filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]
    expiry_days = set()
    

    for ticker in filtered_by_ticker['Ticker']:
        try:
          match = ticker.split(index)[1][:5]
          day_str = match[:2]
          month_str = match[2:]
          
          month_number = months[month_str]
          if month_number < months[input_date_obj.month]:
                    adjusted_year = year + 1
              else:
                    adjusted_year = year
          
          expiry_date = datetime(adjusted_year,months[month_str],int(day_str) )
#           if(input_date_obj.month==months[month_str]):
#                 expiry_days.add(expiry_date)
#           elif((expiry_date-input_date_obj).days >=0):
#                 expiry_days.add(expiry_date)
                
         # if we want just next
          if((expiry_date-input_date_obj).days >=0):
                expiry_days.add(expiry_date)
        except ValueError:
            continue


    return sorted(expiry_days)


# Dictionary to store expiry dates for each date
all_expiry_dates = {}

# Loop through each existing date and fetch expiry dates
for date in existing_dates:
    expiries = get_all_expiry_dates(data, date, index, segment)
    all_expiry_dates[date] = [expiry_date.strftime("%Y-%m-%d") for expiry_date in expiries]

# Printing all expiry dates for each date
print("\nAll Expiry Dates:")
for date, expiries in all_expiry_dates.items():
    print(f"{date}: {expiries}") 

IndentationError: unexpected indent (582487190.py, line 28)

In [10]:
from datetime import datetime
import pandas as pd
import re

months = {
    "JAN": 1, "FEB": 2, "MAR": 3, "APR": 4,
    "MAY": 5, "JUN": 6, "JUL": 7, "AUG": 8,
    "SEP": 9, "OCT": 10, "NOV": 11, "DEC": 12
}

def get_all_expiry_dates(data, input_date, index, segment):
    input_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
    year = input_date_obj.year
    regex_pattern = f"{index}(\d{{2}}).*?\.{segment}"
    filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]
    expiry_days = set()

    for ticker in filtered_by_ticker['Ticker']:
        try:
            match = re.split(f"{index}", ticker)[1][:5]
            day_str = match[:2]
            month_str = match[2:]
            month_number = months[month_str]

            if month_number >= input_date_obj.month:
                adjusted_year = year
            else:
                adjusted_year = year + 1
            
            expiry_date = datetime(adjusted_year, month_number, int(day_str))
            if (expiry_date - input_date_obj).days >= 0:
                expiry_days.add(expiry_date)
        except ValueError:
            continue

    return sorted(expiry_days)

# Assuming 'existing_dates' is a list of dates you have.
# Sample data for the 'data' DataFrame, 'index', and 'segment' need to be defined.
all_expiry_dates = {}
for date in existing_dates:
    expiries = get_all_expiry_dates(data, date, index, segment)
    all_expiry_dates[date] = [expiry_date.strftime("%Y-%m-%d") for expiry_date in expiries]

print("\nAll Expiry Dates:")
for date, expiries in all_expiry_dates.items():
    print(f"{date}: {expiries}")


/var/folders/xm/_6m30k1j1kbgb2ps1nz6t4d00000gn/T/ipykernel_79498/3012367161.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]



All Expiry Dates:
2023-04-03: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-04: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-05: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-06: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-10: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-11: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-12: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-13: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-14: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-17: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-18: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-19: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-20: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-21: ['2023-04-23', '2023-05-23', '2023-06-23']
2023-04-24: ['2023-05-23', '2023-06-23']
2023-04-25: ['2023-05-23', '2023-06-23']
2023-04-26: ['2023-05-23', '2023-06-23']
2023-04-27: ['2023-05-23', '2023-06-23']
2023-04-28: ['2023-05-23', '2023-06-23']


In [ ]:
from datetime import datetime
import pandas as pd

def compute_matching_dates(existing_dates, DTE, data, index, segment, expiry_type):
    # Convert string dates to datetime objects
    date_objects = [datetime.strptime(date, "%Y-%m-%d") for date in existing_dates]
    matched_dates = []
    
    for i, current_date in enumerate(date_objects):
        # Find the target date by moving DTE steps in the existing_dates
        if i + DTE < len(date_objects):
            target_date = date_objects[i + DTE]
        else:
            # If there are not enough days left in the list to add DTE, skip to next date or handle as needed
            continue

        expiries = get_all_expiry_dates(data, current_date.strftime("%Y-%m-%d"), index, segment)

        if expiry_type.lower() == "monthly":
            # Monthly expiries: nearest expiry of the month
            monthly_expiries = [exp for exp in expiries if exp.month == target_date.month]
            if monthly_expiries and min(monthly_expiries, key=lambda d: abs(d - target_date)) == target_date:
                matched_dates.append(target_date.strftime("%Y-%m-%d"))
        elif expiry_type.lower() == "weekly":
            # Weekly expiries: last expiry of the month
            monthly_expiries = [exp for exp in expiries if exp.month == target_date.month]
            if monthly_expiries and max(monthly_expiries) == target_date:
                matched_dates.append(target_date.strftime("%Y-%m-%d"))

    return matched_dates

# Example usage assuming existing_dates and data are defined
existing_dates = ["2023-04-03", "2023-04-04", "2023-04-05", "2023-04-06", "2023-04-10", "2023-04-11", "2023-04-12", "2023-04-13", "2023-04-14", "2023-04-17", "2023-04-18", "2023-04-19", "2023-04-20", "2023-04-21", "2023-04-24", "2023-04-25", "2023-04-26", "2023-04-27", "2023-04-28"]
DTE = 3  # Days to expiry
data = pd.DataFrame(...)  # Define this as per your actual data
index = "IndexSymbol"
segment = "FUT"

matching_dates = compute_matching_dates(existing_dates, DTE, data, index, segment, 'monthly')
print("Matching Dates:")
print(matching_dates)


In [ ]:
Things we have 
1.) existing_dates array which has all dates possible
2.) get_all_expiry_dates() function to get {date}:{expiry} (multiple expiries)
    example for date 14 feb in have 
    14feb [15feb,22feb,1mar,7mar,14mar...]

now we take user input related to "Type of Expiry" and "DTE"

for loop for all existing_dates

Case 1: Type of Expiry is Daily with DTE=x (x is an integer)
    In this case i will first find "daily expiry" from get_all_expiry_dates() 
    i.e. first element corrosponding to expiry in {date}:{expiry} pair then after this
        example if expiries are on [17feb,24feb] and i am on date 11feb,
        then my "daily expiry" will be on 17feb as it is first element 
    I will itrate in my existing_dates array from existing_dates index we are at from for loop (assume it j)
    till index (j+DTE(x)) and date at this value is target_index, also see target_index < len(existing_dates)
       if (target_index==daily expiry){store this date and then move to next date in our existing_dates}
        else(move to next date in our existing_dates)
        
Case 2: Type of Expiry is Monthly-y (y is an integer) with DTE=x (x is an integer)
    In this case i will first find "Monthly-y expiry" from get_all_expiry_dates() 
    example for date 11feb i have expiry stored like [17feb,24feb,1mar,7mar,14mar,21mar,28mar..]
    assume expiry is Monthly-1 then our "Monthly-1 expiry" will bw 24feb
    if expiry is Monthly-2 then our "Monthly-2 expiry" will bw 28mar
    I will itrate in my existing_dates array from existing_dates index we are at from for loop (assume it j)
    till index (j+DTE(x)) and date at this value is target_index, also see target_index < len(existing_dates)
       if (target_index==daily expiry){store this date and then move to next date in our existing_dates}
        else(move to next date in our existing_dates)
    
    


In [ ]:
# Example usage
type_of_expiry = "Daily"
DTE = 2

for j, date in enumerate(existing_dates):
    expiries = get_all_expiry_dates(date)
    if not expiries:
        continue
    
    # Converting the nearest expiry date to datetime object
    daily_expiry = datetime.strptime(expiries[0], "%Y-%m-%d")
    
    # Calculating the target index date based on DTE and convert to datetime.
    target_index = j + DTE
    if target_index < len(existing_dates):
        target_date_str = existing_dates[target_index]
        target_date = datetime.strptime(target_date_str, "%Y-%m-%d")
        
        if target_date == daily_expiry:
            print(f"Base Date: {date}, Daily Expiry: {daily_expiry.strftime('%Y-%m-%d')}, DTE Match Date: {target_date.strftime('%Y-%m-%d')}")
        else:
            continue 

In [ ]:
# Get user input for type of expiry and DTE
type_of_expiry = input("Enter type of expiry")
DTE = int(input("Enter DTE (Days to Expiry) as an integer: "))

 for j, date in enumerate(existing_dates):
    date_dt = datetime.strptime(date, "%Y-%m-%d")
    expiries = get_all_expiry_dates(date)
    
    if type_of_expiry == "Daily":
        expiries = get_all_expiry_dates(date)
        if not expiries:
            continue

        # Converting the nearest expiry date to datetime object
        daily_expiry = datetime.strptime(expiries[0], "%Y-%m-%d")

        # Calculating the target index date based on DTE and convert to datetime.
        target_index = j + DTE
        if target_index < len(existing_dates):
            target_date_str = existing_dates[target_index]
            target_date = datetime.strptime(target_date_str, "%Y-%m-%d")

            if target_date == daily_expiry:
                print(f"Base Date: {date}, Daily Expiry: {daily_expiry.strftime('%Y-%m-%d')}, DTE Match Date: {target_date.strftime('%Y-%m-%d')}")
            else:
                continue
    elif type_of_expiry == 'Monthly':
        # Extract x from "Monthly-x" and determine target month and year
        x = int(type_of_expiry.split('-')[1])
        target_month = (date_dt.month + x - 1) % 12 or 12
        target_year = date_dt.year + (date_dt.month + x - 1) // 12
        
        # Filter expiry dates to find those in the target month and year
        filtered_exp_dates = [d for d in expiries if d.month == target_month and d.year == target_year]
        
        # Calculating the target index date based on DTE and convert to datetime.
        target_index = j + DTE
        if target_index < len(existing_dates):
            target_date = max(filtered_exp_dates)
            target_date = datetime.strptime(target_date_str, "%Y-%m-%d")

            if target_date == latest_expiry_date:
                print(f"Base Date: {date}, Monthly-{x} Expiry: {latest_expiry_date.strftime('%Y-%m-%d')}, DTE Match Date: {target_date.strftime('%Y-%m-%d')}")
            else:
                print(f"Base Date: {date}, No DTE Match for Monthly-{x}")

In [1]:
## below part 

In [ ]:
# Getting user input for type of expiry and DTE
type_of_expiry = input("Enter type of expiry (Daily or Monthly-x, e.g., Monthly-3): ")
DTE = int(input("Enter DTE (Days to Expiry) as an integer: "))

for j, date in enumerate(existing_dates):
    date_dt = datetime.strptime(date, "%Y-%m-%d")
    expiries = [datetime.strptime(d, "%Y-%m-%d") for d in get_all_expiry_dates(date)]
    
    if type_of_expiry == "Daily":
        if not expiries:
            continue

        # Converting the nearest expiry date to datetime object
        daily_expiry = expiries[0]

#         # Calculating the target index date based on DTE and convert to datetime.
#         target_index = j + DTE
#         if target_index < len(existing_dates):
#             target_date_str = existing_dates[target_index]
#             target_date = datetime.strptime(target_date_str, "%Y-%m-%d")

            if target_date == daily_expiry:
                print(f"Base Date: {date}, Daily Expiry: {daily_expiry.strftime('%Y-%m-%d')}, DTE Match Date: {target_date.strftime('%Y-%m-%d')}")
            else:
                    print(f"Base Date: {date}, No DTE Match for Daily")


    elif "Monthly" in type_of_expiry:
        x = int(type_of_expiry.split('-')[1])
        target_month = (date_dt.month + x - 1) % 12 or 12
        target_year = date_dt.year + (date_dt.month + x - 1) // 12

        # Filter expiry dates to find those in the target month and year
        filtered_exp_dates = [d for d in expiries if d.month == target_month and d.year == target_year]
        
        if filtered_exp_dates:
            latest_expiry_date = max(filtered_exp_dates)

            # Calculating the target index date based on DTE and convert to datetime.
            target_index = j + DTE
            if target_index < len(existing_dates):
                target_date_str = existing_dates[target_index]
                target_date = datetime.strptime(target_date_str, "%Y-%m-%d")

                if target_date == latest_expiry_date:
                    print(f"Base Date: {date}, Monthly-{x} Expiry: {latest_expiry_date.strftime('%Y-%m-%d')}, DTE Match Date: {target_date.strftime('%Y-%m-%d')}")
                else:
                    print(f"Base Date: {date}, No DTE Match for Monthly-{x}")


In [ ]:
# Function to get ITM/OTM level dynamically
def get_strike_level(strike_selection):
    import re
    if strike_selection == 'ATM':
        return 0, 'ATM'
    match = re.match(r'(ITM|OTM)-(\d+)', strike_selection)
    return int(level), strike_type


segment_data = pd.DataFrame({
    'segment': ['NIFTY', 'BANKNIFTY', 'BANKEX', 'SENSEX', 'CRUDEOIL', 'SILVER', 'GOLD', 'NICKEL', 'ZINC'],
    'lot_size': [25, 15, 15, 10, 100, 30, 100, 1500, 5000], # it changes
    'strike_diff': [50, 100, 100, 100, 50, 50, 50, 50, 50]
})

def get_all_expiry_dates(data, input_date, index, option_type, segment, selected_strike):
    
    # Parsing the input date to extract the month and year
    input_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
    year = input_date_obj.year
    month_abbr = input_date_obj.strftime("%b").upper()  
    
    # Constructing the ticker pattern
    ticker_pattern = f"{index}d{{2}}{month_abbr}{selected_strike}{option_type}.{segment}"

    # Filtering tickers matching the pattern
    filtered_tickers = data[data['Ticker'].str.contains(ticker_pattern, regex=True)]

    # Extracting unique days from the filtered tickers
    expiry_days = set()
    for ticker in filtered_tickers['Ticker']:
        # Extracting the day (2 digits before the month abbreviation)
        day_str = ticker.split(index)[1][:2]
        try:
            expiry_days.add(int(day_str))  # Ensure it's a valid integer
        except ValueError:
            continue

    # Combining days with the input month and year
    expiry_dates = [datetime(year, input_date_obj.month, day) for day in sorted(expiry_days)]

    return expiry_dates


# also include to take input of "what kind of expiry i want" and "DTE" {Days to expiry}
# "what kind of expiry i want" means daily/weekly/monthly
def leg_function(data, index, action, option_type, lots, strike_selection, 
                entry_time, universal_exit_time, stop_loss, stop_loss_type, target, target_type, 
                segment_data):
    """
    Function to determine trade details for options based on market data and parameters.
    """
    import pandas as pd
    
    # Getting strike difference for the "segment_data"
    strike_diff = segment_data.loc[segment_data['segment'] == index, 'strike_diff'].iloc[0]
    
    # Constructing the ticker name from index and Segment
    ticker_name = f"{index}-I.{Segment}"

    # Get the entry spot price by finding the close price at the entry time for the given ticker
    entry_data = data[(data['symbol'] == ticker_name) & (data['time'] == entry_time)]
    entry_spot = float(entry_data['close'].iloc[0])
    
    # Calculating ATM strike (round spot price to nearest strike difference multiple)
    atm_strike = round(entry_spot / strike_diff) * strike_diff
    
    # Calculate the selected strike by use of "get_strike_level()" function
    level, strike_type = get_strike_level(strike_selection)
    if strike_type == 'ITM':
        multiplier = -1 if option_type == 'CE' else 1
    elif strike_type == 'OTM':
        multiplier = 1 if option_type == 'CE' else -1
    selected_strike = atm_strike + (multiplier * level * strike_diff)
    
    # Getting all possible expiry dates in an array
    expiry_dates = get_all_expiry_dates(data, input_date, index, option_type, segment, selected_strike)
    
    # From all element in "get_all_expiry_dates" we will get the required date and get "spot"
    # see if expiry is weekly , monthly or daily
    
    # Calculating stop loss and target levels
    stop_loss_level = spot * stop_loss_type if stop_loss == 'yes' else None
    target_level = spot * target_type if target == 'yes' else None
    
    # Filtering data for monitoring (from entry time onwards)
    # we will observe that "date"{from input_date} and "Ticker"{correct} and we will get "monitoring_data"

    monitoring_data = data[
        (data['symbol'] == index) & 
        (data['time'] >= entry_time) & 
        (data['time'] <= universal_exit_time)
    ].sort_values('time')
    
    # Initializing exit variables
    exit_spot = None
    exit_reason = None
    actual_exit_time = None
    
    # Monitoring price movement to find exit point
    for idx, row in monitoring_data.iterrows():
        current_spot = float(row['close'])
        current_time = row['time']
        
        # Checking stop loss
        if stop_loss == 'yes':
            if ((action == 'BUY' and current_spot <= stop_loss_level) or 
                (action == 'SELL' and current_spot >= stop_loss_level)):
                exit_spot = current_spot
                exit_reason = 'Stop Loss'
                actual_exit_time = current_time
                break
        
        # Checking target
        if target == 'yes':
            if ((action == 'BUY' and current_spot >= target_level) or 
                (action == 'SELL' and current_spot <= target_level)):
                exit_spot = current_spot
                exit_reason = 'Target'
                actual_exit_time = current_time
                break
        
        # Checking universal exit time
        if current_time == universal_exit_time:
            exit_spot = current_spot
            exit_reason = 'Time Exit'
            actual_exit_time = current_time
            break
    
    # If no exit condition was met before universal_exit_time, use the last available price
    if exit_spot is None:
        last_data = monitoring_data.iloc[-1]
        exit_spot = float(last_data['close'])
        exit_reason = 'Time Exit'
        actual_exit_time = last_data['time']
    
    # Calculate PnL
    pnl = (exit_spot - entry_spot) * lots if action == 'BUY' else -(exit_spot - entry_spot) * lots
    
    # quantity insted of lots {remove it}
    pnl = (exit_spot - entry_spot) * lots if action == 'BUY' else -(exit_spot - entry_spot) * lots
    
    # Return all calculated values
    trade_details = {
        'entry_time': entry_time,
        'entry_spot': entry_spot,
        'atm_strike': atm_strike,
        'selected_strike': selected_strike,
        'option_type': option_type,
        'action': action,
        'lots': lots,
        'stop_loss_level': stop_loss_level,
        'target_level': target_level,
        'exit_time': actual_exit_time,
        'exit_spot': exit_spot,
        'exit_reason': exit_reason,
        'universal_exit_time': universal_exit_time,
        'pnl': pnl
    }
    
    return trade_details

# Leg 1
leg1 = leg_function(
    data=data,
    Index='GOLD',
    action='BUY',
    option_type='CE',
    lots=1,
    strike_selection='ITM-1',
    entry_time='09:00',
    universal_exit_time='10:15',
    stop_loss='yes',
    stop_loss_type=0.97,
    target='yes',
    target_type=1.05,
    segment_data=segment_data
)

print(f"Leg 1: {leg1}")

# Leg 2 (entry time is the exit time of leg 1 if within universal_exit_time)
if leg1['exit_time'] < '10:15':  # Universal exit time
    leg2 = leg_function(
        data=data,
        Index='GOLD',
        action='SELL',
        option_type='PE',
        lots=2,
        strike_selection='OTM-1',
        entry_time=leg1['exit_time'],  # Exit time of leg 1
        universal_exit_time='10:15',
        stop_loss='yes',
        stop_loss_type=0.95,
        target='yes',
        target_type=1.04,
        segment_data=segment_data
        # expiry type
        # DTE
    ) 
    print(f"Leg 2: {leg2}")

# Leg 3 (manually chosen entry time)
leg3 = leg_function(
    data=data,
    Index='GOLD',
    action='BUY',
    option_type='CE',
    lots=1,
    strike_selection='ATM',
    entry_time='10:00',
    universal_exit_time='10:15',
    stop_loss='yes',
    stop_loss_type=0.96,
    target='yes',
    target_type=1.06,
    segment_data=segment_data
)

print(f"Leg 3: {leg3}")


In [ ]:
# Function to compute matching dates based on DTE using only existing_dates
def compute_matching_dates(existing_dates, DTE, data, index, segment):
    matched_dates = {}
    date_objs = [datetime.strptime(date, "%Y-%m-%d") for date in existing_dates]

    for i, current_date in enumerate(date_objs):
        target_index = i + DTE
        if target_index < len(date_objs):
            target_date = date_objs[target_index]
            expiries = get_all_expiry_dates(data, existing_dates[i], index, segment)
            if target_date in expiries:
                if existing_dates[i] not in matched_dates:
                    matched_dates[existing_dates[i]] = []
                matched_dates[existing_dates[i]].append(target_date.strftime("%Y-%m-%d"))

    return matched_dates

# Example usage:
existing_dates = ['2023-04-03', '2023-04-04', '2023-04-05', '2023-04-06', '2023-04-10']
data = pd.DataFrame({'Ticker': ['INDEX23APR.FUT', 'INDEX23MAY.FUT', 'INDEX23JUN.FUT']})
index = 'INDEX'
segment = 'FUT'
DTE = 3  # Days to Expiry, adjusting as per your scenario

matching_dates = compute_matching_dates(existing_dates, DTE, data, index, segment)
print("\nMatching Dates:")
for date, expiries in matching_dates.items():
    print(f"{date}: {expiries}")


# b

In [4]:
from datetime import datetime
import pandas as pd
import re  

months = {
    "JAN": 1, "FEB": 2, "MAR": 3, "APR": 4,
    "MAY": 5, "JUN": 6, "JUL": 7, "AUG": 8,
    "SEP": 9, "OCT": 10, "NOV": 11, "DEC": 12
}

def get_all_expiry_dates(data, input_date, index, segment):
    input_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
    year = input_date_obj.year
    regex_pattern = f"{index}(\d{{2}}).*?\.{segment}"
    filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]
    expiry_days = set()
    
    for ticker in filtered_by_ticker['Ticker']:
        try:
            match = re.search(f"{index}(\d{{2}}{month})(\d{{2}}).*\.{segment}", ticker, re.IGNORECASE)
            if match:
                day_str = match.group(2)
                month_str = match.group(1)[-3:]  # Last three characters will be the month in the format we defined

                # Adjust year based on month crossing
                month_number = months[month_str]
                if month_number < input_date_obj.month:
                    adjusted_year = year + 1
                else:
                    adjusted_year = year
                
                expiry_date = datetime(adjusted_year, month_number, int(day_str))
                if (expiry_date - input_date_obj).days >= 0:
                    expiry_days.add(expiry_date)
        except ValueError:
            continue

    return sorted(expiry_days)

# Usage of the function, assumes data, existing_dates, index, and segment variables are defined
all_expiry_dates = {}

# Loop through each existing date and fetch expiry dates
for date in existing_dates:
    expiries = get_all_expiry_dates(data, date, index, segment)
    all_expiry_dates[date] = [expiry_date.strftime("%Y-%m-%d") for expiry_date in expiries]

# Printing all expiry dates for each date
print("\nAll Expiry Dates:")
for date, expiries in all_expiry_dates.items():
    print(f"{date}: {expiries}")


/var/folders/xm/_6m30k1j1kbgb2ps1nz6t4d00000gn/T/ipykernel_79498/696991703.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]


NameError: name 'month' is not defined

In [5]:
def get_all_expiry_dates(data, input_date, index, segment):
    input_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
    year = input_date_obj.year
    # Correct the regex pattern to properly capture two digits for month and day.
    regex_pattern = f"{index}(\\d{{2}})(\\d{{2}}).*\\.{segment}"
    filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]
    expiry_days = set()
    
    for ticker in filtered_by_ticker['Ticker']:
        try:
            # Use re.search to capture the day and month directly from the ticker based on the corrected pattern.
            match = re.search(f"{index}(\\d{{2}})(\\d{{2}}).*\\.{segment}", ticker, re.IGNORECASE)
            if match:
                day_str = match.group(2)  # Second group: day
                month_str = match.group(1)  # First group: month, assuming it's in the correct position
                
                month_number = months[month_str[-3:]]  # Ensure the month_str is appropriately extracted
                # Determine if we need to adjust the year based on the month.
                if month_number < input_date_obj.month:
                    adjusted_year = year + 1
                else:
                    adjusted_year = year
                
                # Calculate the expiry date with the adjusted year.
                expiry_date = datetime(adjusted_year, month_number, int(day_str))
                # Only add expiry dates that are in the future relative to the input date.
                if (expiry_date - input_date_obj).days >= 0:
                    expiry_days.add(expiry_date)
        except ValueError:
            continue

    return sorted(expiry_days)

# Usage of the function, assumes data, existing_dates, index, and segment variables are defined
all_expiry_dates = {}

# Loop through each existing date and fetch expiry dates
for date in existing_dates:
    expiries = get_all_expiry_dates(data, date, index, segment)
    all_expiry_dates[date] = [expiry_date.strftime("%Y-%m-%d") for expiry_date in expiries]

# Printing all expiry dates for each date
print("\nAll Expiry Dates:")
for date, expiries in all_expiry_dates.items():
    print(f"{date}: {expiries}")


/var/folders/xm/_6m30k1j1kbgb2ps1nz6t4d00000gn/T/ipykernel_79498/2991858906.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]



All Expiry Dates:
2023-04-03: []
2023-04-04: []
2023-04-05: []
2023-04-06: []
2023-04-10: []
2023-04-11: []
2023-04-12: []
2023-04-13: []
2023-04-14: []
2023-04-17: []
2023-04-18: []
2023-04-19: []
2023-04-20: []
2023-04-21: []
2023-04-24: []
2023-04-25: []
2023-04-26: []
2023-04-27: []
2023-04-28: []


In [1]:
from datetime import datetime
import pandas as pd
import re

# Dictionary mapping abbreviated month names to their numeric equivalents.
months = {
    "JAN": 1, "FEB": 2, "MAR": 3, "APR": 4,
    "MAY": 5, "JUN": 6, "JUL": 7, "AUG": 8,
    "SEP": 9, "OCT": 10, "NOV": 11, "DEC": 12
}

# Function to find all expiry dates for given data, date, index, and segment.
def get_all_expiry_dates(data, input_date, index, segment):
    # Converting input date from string to datetime object.
    input_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
    # Extracting the year from the input date.
    year = input_date_obj.year
    # Creating a regex pattern to filter tickers by index and segment.
    regex_pattern = f"{index}(\d{{2}}).*?\.{segment}"
    # Filtering data based on the regex pattern.
    filtered_by_ticker = data[data['Ticker'].str.contains(regex_pattern, case=False, regex=True)]
    # Setting to hold unique expiry dates.
    expiry_days = set()
    
    # Looping through each ticker in the filtered data.
    for ticker in filtered_by_ticker['Ticker']:
        try:
            # Splitting the ticker by the index and extract the day and month part.
            match = ticker.split(index)[1][:5]
            day_str = match[:2]
            month_str = match[2:]
            
            # Determining the month number from the abbreviation.
            month_number = months[month_str]
            # Adjusting the year based on whether the ticker's month is before the input date's month.
            if month_number < input_date_obj.month:
                adjusted_year = year + 1
            else:
                adjusted_year = year
            
            # Creating the expiry date from the adjusted year and parsed day and month.
            expiry_date = datetime(adjusted_year, month_number, int(day_str))
            
            # Adding expiry date to the set if it's in the future relative to the input date.
            if (expiry_date - input_date_obj).days >= 0:
                expiry_days.add(expiry_date)
        except ValueError:
            # Continue loop if an error occurs (e.g., invalid date parsing).
            continue

    # Returning a sorted list of unique expiry dates.
    return sorted(expiry_days)

# Dictionary to store expiry dates for each input date.
all_expiry_dates = {}

# Looping through each date in the list of existing dates.
for date in existing_dates:
    # Getting expiry dates for the current date using the function defined above.
    expiries = get_all_expiry_dates(data, date, index, segment)
    # Formatting expiry dates and store them in the dictionary.
    all_expiry_dates[date] = [expiry_date.strftime("%Y-%m-%d") for expiry_date in expiries]

# Printing all expiry dates for each input date.
print("\nAll Expiry Dates:")
for date, expiries in all_expiry_dates.items():
    print(f"{date}: {expiries}")


NameError: name 'existing_dates' is not defined

In [25]:
# Re-importing the necessary module and redefining the function as the execution state was reset
import re

# Load the dataset again
data_path = '/mnt/data/GFDLMCX_BACKADJUSTED_03042023 2.csv'
data = pd.read_csv(data_path)

# Redefine the function to specifically target CRUDEOIL index and MCX segment with adjusted criteria
def extract_crudeoil_date_month(ticker):
    # Check for specific index and segment
    if "CRUDEOIL" in ticker and ticker.endswith(".MCX"):
        # The expected format is CRUDEOIL{date}{month}{selected_strike}{option_type}.MCX
        # Example: CRUDEOIL23APR4500PE.MCX
        # Strip away the index and segment parts to isolate the middle portion
        main_part = ticker.replace("CRUDEOIL", "").replace(".MCX", "")
        # Use regex to find the pattern {date}{month}{selected_strike}{option_type}
        match = re.search(r'(\d{2})([A-Z]{3})\d{4}(CE|PE)', main_part)
        if match:
            # Extract the date and month from the match groups
            date = match.group(1)
            month = match.group(2)
            return date, month
    return None

# Apply the adjusted function to all tickers in the dataset
crudeoil_date_month = [extract_crudeoil_date_month(ticker) for ticker in data['Ticker']]
# Get unique date and month combinations
unique_crudeoil_date_month = set([dm for dm in crudeoil_date_month if dm is not None])
unique_crudeoil_date_month


{('23', 'APR'), ('23', 'JUN'), ('23', 'MAY')}

In [26]:
# Updated function to extract and verify date-month pairs against a list of existing dates
def extract_date_month(ticker, index, segment, existing_dates):
    if ticker.startswith(index) and ticker.endswith(segment):
        main_part = ticker[len(index):-len(segment)-1]  # Exclude the dot in the segment
        match = re.search(r'(\d{2})([A-Z]{3})\d{4}(CE|PE)', main_part)
        if match:
            day = int(match.group(1))
            month = match.group(2)
            year = datetime.now().year  # Assuming current year; adjust as necessary
            ticker_date_str = f"{year}-{month}-{day:02d}"
            ticker_date = datetime.strptime(ticker_date_str, "%Y-%b-%d")
            
            # Check if ticker_date is in the list of existing dates
            if ticker_date in existing_dates:
                return (day, month)
    return None

In [30]:
import pandas as pd
import re
from datetime import datetime

# Load the dataset
data_path = 'GFDLMCX_BACKADJUSTED_03042023 2.csv'
data = pd.read_csv(data_path)


# Generalized function to extract date and month from a ticker string based on given index and segment
def extract_date_month(ticker, index, segment):
    # Format is {index}{date}{month}{selected_strike}{option_type}.{segment}
    # Example: CRUDEOIL23APR4500PE.MCX
    if ticker.startswith(index) and ticker.endswith(segment):
        # Remove index and segment to isolate the middle portion
        main_part = ticker[len(index):-len(segment)-1]  # Exclude the dot in the segment
        # Regex to find the pattern {date}{month}{selected_strike}{option_type}
        match = re.search(r'(\d{2})([A-Z]{3})\d{4}(CE|PE)', main_part)
        if match:
            # Extract the date and month
            date = match.group(1)
            month = match.group(2)
            return date, month
    return None

# Example usage: extracting dates and months for CRUDEOIL tickers in the MCX segment
date_month_info = [extract_date_month(ticker, index, segment) for ticker in data['Ticker']]
unique_date_month_info = set([info for info in date_month_info if info is not None])

# Displaying unique date and month combinations
print(unique_date_month_info)


{('23', 'MAY'), ('23', 'APR')}


In [31]:
import pandas as pd
import re
from datetime import datetime

# Generalized function to extract date and month from a ticker string based on given index, segment, and date check
def extract_date_month(ticker, index, segment, existing_dates):
    # Format is {index}{date}{month}{selected_strike}{option_type}.{segment}
    # Example: CRUDEOIL23APR4500PE.MCX
    if ticker.startswith(index) and ticker.endswith(segment):
        # Remove index and segment to isolate the middle portion
        main_part = ticker[len(index):-len(segment)-1]  # Exclude the dot in the segment
        # Regex to find the pattern {date}{month}{selected_strike}{option_type}
        match = re.search(r'(\d{2})([A-Z]{3})\d{4}(CE|PE)', main_part)
        if match:
            # Extract the date and month
            day = int(match.group(1))
            month = match.group(2)
            year = datetime.now().year  # Assuming current year; adjust as necessary if required
            # Create a datetime object to compare against existing_dates
            extracted_date = datetime.strptime(f"{day} {month} {year}", "%d %b %Y")

            # Check if extracted_date is in the list of existing dates
            if extracted_date in existing_dates:
                return day, month
    return None

date_month_info = [extract_date_month(ticker, index, segment, existing_dates) for ticker in data['Ticker']]
unique_date_month_info = set([info for info in date_month_info if info is not None])

# Displaying unique date and month combinations that match the existing dates
print(unique_date_month_info)


set()


now we will create a function which take date which Exists, segment, index, expiry type, DTE {for monthly/weekly} 

if daily expiry then see ticker named 




filtered_date_to_work = f(existing_date, type_of_expiry, DTE)

if type_of_expiry is daily:
   Date is existing_date and Ticker Date consists of string existing_date {in daily DTE is not defined}
   
if type_of_expiry is weekly:
   put that existing_date inside a function which tell me all existing expiry of that month then see what week we need to work like week 1 , week 2 then we will also take DTE as input also i.e. week 1 DTE 1 means we need to go to week 1 then go to 

all_expiry_of_month = f(one of existing_date)
   This function tell me all expiry of this month
   
Make a List of Existing Date
2023-04-03: Exists
2023-04-04: Exists
2023-04-05: Exists
2023-04-06: Exists
2023-04-07: Does not exist
2023-04-08: Does not exist
2023-04-09: Does not exist

and convert to 
2023-04-03: Exists
2023-04-04: Exists
2023-04-05: Exists
2023-04-06: Exists

Take input of type_of_expiry, DTE

example:
  if type_of_expiry is Daily then we will see our all_expiry_of_month and find nearest element to this date ... "date in expiry - daily date" should be min ...store that date in ticker_date list
  
  if type_of_expiry is Weekly we will do a + DTE to the input Date in our List and see if the Date we get to is same as that week expiry or Not, if not we will move to next possible existing date else if we we will store that date in ticker_date list

